#Part 1 - Data Processing


In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import cleantext
import matplotlib.pyplot as plt

# Task 1:
url = "https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv"
df = pd.read_csv(url)

def clean_content(text):
    cleaned_text = cleantext.clean(text,
                                   lower=True,
                                   no_urls=True,
                                   no_emails=True,
                                   no_phone_numbers=True,
                                   no_numbers=True,
                                   no_digits=True,
                                   no_currency_symbols=True,
                                   no_punct=True,
                                   replace_with_url="<URL>",
                                   replace_with_email="<EMAIL>",
                                   replace_with_phone_number="<PHONE>",
                                   replace_with_number="<NUM>",
                                   replace_with_digit="<NUM>",
                                   replace_with_currency_symbol="<CUR>")
    return cleaned_text

df['clean_content'] = df['content'].apply(clean_content)

# Tokenize the text
df['tokens'] = df['clean_content'].apply(word_tokenize)

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['tokens_no_stopwords'] = df['tokens'].apply(lambda tokens: [token for token in tokens if token not in stop_words])

# Stemming
ss = SnowballStemmer('english')
df['tokens_stemmed'] = df['tokens_no_stopwords'].apply(lambda tokens: [ss.stem(token) for token in tokens])

def compute_vocabulary_size(tokens_list):
    unique_words = set()
    for tokens in tokens_list:
        unique_words.update(tokens)
    return len(unique_words)

# Compute vocabulary size before removing stopwords
original_vocabulary_size = compute_vocabulary_size(df['tokens'])
print(f"Original vocabulary size: {original_vocabulary_size}")

# Compute vocabulary size after removing stopwords
reduced_vocabulary_size_no_stopwords = compute_vocabulary_size(df['tokens_no_stopwords'])
print(f"Vocabulary size after removing stopwords: {reduced_vocabulary_size_no_stopwords}")

# Compute reduction rate of the vocabulary size after removing stopwords
reduction_rate_no_stopwords = (original_vocabulary_size - reduced_vocabulary_size_no_stopwords) / original_vocabulary_size * 100
print(f"Reduction rate after removing stopwords: {reduction_rate_no_stopwords:.2f}%")

# Compute vocabulary size after stemming
reduced_vocabulary_size_stemmed = compute_vocabulary_size(df['tokens_stemmed'])
print(f"Vocabulary size after stemming: {reduced_vocabulary_size_stemmed}")

# Compute reduction rate of the vocabulary size after stemming
reduction_rate_stemmed = (original_vocabulary_size - reduced_vocabulary_size_stemmed) / original_vocabulary_size * 100
print(f"Reduction rate after stemming: {reduction_rate_stemmed:.2f}%")






In [ ]:
# Task 2: 
import re
import numpy as np
def count_elements(row):
    # Combine all text columns
    combined_text = ' '.join(row.astype(str))

    # Counts URLs
    url_pattern = r'(?:http[s]?://|www\.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' 
    urls = re.findall(url_pattern, combined_text)
    url_count = len(urls)

    # Counts dates
    date_pattern = r'\d{4}-\d{2}-\d{2}'
    dates = re.findall(date_pattern, combined_text)
    date_count = len(dates)

    # Counts numeric values
    numeric_pattern = r'\d'
    numeric_values = re.findall(numeric_pattern, combined_text)
    numeric_count = len(numeric_values)

    return url_count, date_count, numeric_count

df['url_count'], df['date_count'], df['numeric_count'] = zip(*df.apply(count_elements, axis=1))

total_urls = df['url_count'].sum()
total_dates = df['date_count'].sum()
total_numeric_values = df['numeric_count'].sum()

print(f"Total number of URLs: {total_urls}")
print(f"Total number of dates: {total_dates}")
print(f"Total number of numeric values: {total_numeric_values}")


# Counts the frequency of the top 20 authors
author_counts = df['authors'].value_counts()

plt.figure(figsize=(12, 6))
author_counts.head(20).plot(kind='bar')
plt.xlabel("Authors")
plt.ylabel("Frequency")
plt.title("Top 20 authors")
plt.show()


# Calculate the number of missing values in each column
missing_values = df.isnull().sum()
sorted_missing_values = missing_values.sort_values(ascending=False)


plt.figure(figsize=(12, 6))
sorted_missing_values.head(6).plot(kind='bar') #We started off by plotting for each column, but only the first 5 (in the sorted) are interesting as the others are 0.
plt.xlabel("Columns")
plt.ylabel("Number of missing values")
plt.title("Missing values in each column")
plt.xticks(rotation=45)
plt.show()

def count_word_frequencies(df, ignore_words=None):
    word_freq = {}
    for tokens in df:
        for word in tokens:
            if ignore_words and word.lower() in ignore_words:
                continue
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1

    items = list(word_freq.items())
    items.sort(reverse=True, key=lambda x: x[1])

    top_word_freq = items[:10000]
    return top_word_freq

# Before removing stopwords and applying stemming
top10000_before = count_word_frequencies(df['tokens'])

# After removing stopwords and applying stemming
top10000_after = count_word_frequencies(df['tokens_stemmed'], ignore_words=['url', 'num', 'email', 'phone', 'cur', '<', '>'])

words_before = [word[0] for word in top10000_before]
freq_before = [freq[1] for freq in top10000_before]

words_after = [word[0] for word in top10000_after]
freq_after = [freq[1] for freq in top10000_after]

# Bar plot of the top 10,000 most frequent words before and after preprocessing
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

ax1.plot(freq_before)
ax1.set_xlabel("Word Index")
ax1.set_ylabel("Frequencies")
ax1.set_title("Before preprocessing")

ax2.plot(freq_after)
ax2.set_xlabel("Word Index")
ax2.set_ylabel("Frequencies")
ax2.set_title("After preprocessing")

plt.show()

# Additional statistics and visualizations

# Distribution of domains
domain_counts = df['domain'].value_counts()

plt.figure(figsize=(12, 6))
domain_counts.plot(kind='bar')
plt.xlabel("Domains")
plt.ylabel("Frequency")
plt.title("Domain frequency")
plt.show()

# Calculates the average number of tokens before and after preprocessing
df['content_length_before'] = df['tokens'].apply(len)
df['content_length_after'] = df['tokens_stemmed'].apply(len)

avg_before = np.mean(df['content_length_before'])
avg_after = np.mean(df['content_length_after'])

print(f"Average number of tokens per article before preprocessing: {avg_before:.2f}")
print(f"Average number of tokens per article after preprocessing: {avg_after:.2f}")

# Create a bar chart to visualize the difference
plt.bar(['Before preprocessing', 'After preprocessing'], [avg_before, avg_after])
plt.ylabel("Average number of tokens")
plt.title("Average content length before and after preprocessing")
plt.show()

# Counts number of articles in each category

type_counts = df['type'].value_counts()
plt.figure(figsize=(12, 6))
type_counts.plot(kind='bar')
plt.xlabel("Article type")
plt.ylabel("Frequency")
plt.title("Distribution of articles by type")
plt.show()




In [ ]:
# Task 3

# print("reading")
# data = pd.read_csv('news_sample.csv', engine="python")

# print("cleaning")
# data['content'] = data['content'].apply(clean_content)

# print ("tokenizing")
# # Tokenize the text
# data['content'] = data['content'].apply(word_tokenize)

# print ("stopwords")
# # Remove stopwords
# data['content'] = data['content'].apply(lambda tokens: [token for token in tokens if token not in stop_words])

# print("stemming")
# # Stemming
# data['content'] = data['content'].apply(lambda tokens: [ss.stem(token) for token in tokens])

# print("to csv")
# data.to_csv('news_sample_cleaned2.csv', index=False)

# Vi laver en ny fil med kun 2 content-kolonner

In [ ]:
df_large = pd.read_csv('news_sample_cleaned2.csv', nrows=10000)

df_large.fillna('', inplace=True)
print(df_large.isna().sum())



In [ ]:
# Task 4
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# This is part of part 2 task 0
df_large['label'] = df_large['type'].apply(lambda x: 'reliable' if x in ['reliable', 'political'] else 'fake')

# This is part of part 2 task 1
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_large['content'])
y = df_large['label']
    
# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)


# Part 2, task 1


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000, C=10, penalty='l2')),
    ('Naive Bayes', MultinomialNB(alpha=0.1)),
    ('Support Vector Machine', LinearSVC(max_iter=1000, C=10, penalty='l2'))
]

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} accuracy: {accuracy:.4f}")





In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the range of hyperparameters to search over
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

# Create a grid search object and fit it to the training data
lr = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Validation accuracy:", grid_search.best_score_)
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Define the hyperparameters for the Naive Bayes classifier
nb_params = {'alpha': [0.1, 0.5, 1.0]}

# Create a Naive Bayes classifier with the hyperparameters
nb = MultinomialNB()
nb_grid_search = GridSearchCV(nb, nb_params, cv=5)
nb_grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("Naive Bayes best hyperparameters:", nb_grid_search.best_params_)
print("Validation accuracy:", nb_grid_search.best_score_)

# Define the hyperparameters for the SVM classifier
svm_params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

# Create an SVM classifier with the hyperparameters
svm = LinearSVC(max_iter=1000)
svm_grid_search = GridSearchCV(svm, svm_params, cv=5)
svm_grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("SVM best hyperparameters:", svm_grid_search.best_params_)
print("Validation accuracy:", svm_grid_search.best_score_)


In [ ]:
# # Clean title
# print("cleaning")
# df_large['title'] = df_large['title'].apply(clean_content)

# print("tokenizing")
# # Tokenize title
# df_large['title'] = df_large['title'].apply(word_tokenize)

# print("stopwords")
# # Remove stopwords from title
# df_large['title'] = df_large['title'].apply(lambda tokens: [token for token in tokens if token not in stop_words])
# print("stemming")
# # Stemming title
# df_large['title'] = df_large['title'].apply(lambda tokens: [ss.stem(token) for token in tokens])

# print("to csv")
# df_large['title'].to_csv('processed_titles.csv', index=False)

df_large_titles = pd.read_csv('processed_titles.csv', nrows=10000)


## Part 2, task 2

In [ ]:
from scipy.sparse import hstack

X_title = vectorizer.fit_transform(df_large_titles['title'])
X_domain = vectorizer.fit_transform(df_large['domain'])
X_keywords = vectorizer.fit_transform(df_large['meta_keywords'])

X_combined = hstack([X, X_keywords])

X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y, test_size=0.2, random_state=42)
X_val_combined, X_test_combined, y_val_combined, y_test_combined = train_test_split(X_test_combined, y_test_combined, test_size=0.5, random_state=42)

for name, model in models:
    model.fit(X_train_combined, y_train_combined)
    y_pred_combined = model.predict(X_test_combined)
    accuracy_combined = accuracy_score(y_test_combined, y_pred_combined)
    print(f"{name} accuracy (combined features): {accuracy_combined:.4f}")
# Den får 100% pga. artiklerne er klassificeret på baggrund af domænet og ikke artikler, overfitter

## Part 3

In [ ]:
# Add necessary imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras import backend as K


def f1_score_custom(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Encode the labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the dataset into training, validation, and test sets
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
X_val_nn, X_test_nn, y_val_nn, y_test_nn = train_test_split(X_test_nn, y_test_nn, test_size=0.5, random_state=42)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
from keras.metrics import Precision, Recall


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), 
              metrics=['accuracy', f1_score_custom])


# Train the model
history = model.fit(X_train.toarray(), y_train_nn, validation_data=(X_val.toarray(), y_val_nn), epochs=10, batch_size=32)

# Test the model
nnscore = model.evaluate(X_test.toarray(), y_test_nn, batch_size=32)
print(f"Neural network accuracy: {nnscore[1]:.4f}")



In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Naive Bayes', MultinomialNB()),
    ('Support Vector Machine', LinearSVC(max_iter=1000))
]
# We show the confusion matrix and f1-score for each simple model
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"{name} accuracy: {accuracy:.4f}")
    print(f"{name} F1-score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()



In [ ]:
# Part 4, task 1
# We computed this value (nnscore) when completing part 3. We saved the result in a variable which we just use here

print(f"Neural network:\nAccuracy: {nnscore[1]:.4f} \nF1 score: {nnscore[2]:.4f} ")

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1score = f1_score(y_test, y_pred, average='weighted')
    print(f"\n{name}: \nAccuracy: {accuracy:.4f} \nF1 score: {f1score:.4f}")


In [ ]:
import pandas as pd

def read_liar_tsv(file_path):
    columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'party_affiliation',
               'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']
    return pd.read_csv(file_path, sep='\t', header=None, names=columns, index_col=False)

train_df = read_liar_tsv('train.tsv')
test_df = read_liar_tsv('test.tsv')
valid_df = read_liar_tsv('valid.tsv')

liar_df = pd.concat([train_df, test_df, valid_df]).reset_index(drop=True)

# Preprocess the LIAR dataset
liar_df['label'] = liar_df['label'].apply(lambda x: 'reliable' if x in ['true', 'mostly-true', 'half-true'] else 'fake')

X_liar = vectorizer.fit_transform(liar_df['statement'])
y_liar = liar_df['label']


y_liar_encoded = le.transform(y_liar)
y_liar_categorical = to_categorical(y_liar_encoded)


from sklearn.metrics import f1_score

for name, model in models:
    y_liar_pred = model.predict(X_liar)
    f1 = f1_score(y_liar, y_liar_pred, pos_label='reliable')
    print(f"{name} F1-score on LIAR dataset: {f1:.4f}")

# Evaluate the advanced model on the LIAR dataset
nn_liar_score = model.evaluate(X_liar.toarray(), y_liar_categorical, batch_size=32)
print(f"Neural network F1-score on LIAR dataset: {nn_liar_score[2]}")
